In [1]:
import datetime, time, os, requests

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3.raw"
}
SEARCH_URL = "https://api.github.com/search/code"
DOWNLOAD_DIR = "outputs/github-ts-output"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def date_ranges(start, end, delta_days=7):
    """Yield (YYYY-MM-DD) tuples from start to end in delta_days increments."""
    cur = start
    while cur < end:
        nxt = min(end, cur + datetime.timedelta(days=delta_days))
        yield cur.strftime("%Y-%m-%d"), (nxt - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        cur = nxt

def search_and_download(start, finish):
    query = f"extension:ts language:TypeScript pushed:{start}..{finish}"
    for page in range(1, 11):  # GitHub caps at 10 pages
        params = {"q": query, "per_page": 100, "page": page}
        r = requests.get(SEARCH_URL, headers=HEADERS, params=params)
        r.raise_for_status()
        items = r.json().get("items", [])
        if not items: break
        for item in items:
            raw = item["html_url"]\
                   .replace("https://github.com/", "https://raw.githubusercontent.com/")\
                   .replace("/blob/", "/")
            resp = requests.get(raw, headers=HEADERS, timeout=10)
            if resp.ok:
                fn = f"{item['repository']['full_name'].replace('/', '_')}__{item['path'].replace('/', '_')}"
                with open(os.path.join(DOWNLOAD_DIR, fn), "w", encoding="utf-8") as f:
                    f.write(resp.text)
        time.sleep(2)

if __name__ == "__main__":
    start_date = datetime.date(2015, 1, 1)
    end_date   = datetime.date.today()
    for s, f in date_ranges(start_date, end_date, delta_days=1):
        search_and_download(s, f)
